In [1]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in c:\users\jshwa\anaconda3\lib\site-packages (3.8.1)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install utils

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jshwa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from matplotlib import font_manager, rc
font_fname = 'c:/windows/fonts/gulim.ttc'     # A font of your choice
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

In [7]:
import torch
from time import time
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [9]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [10]:
def read_data(filename):
    with open(filename, 'r', encoding='UTF8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data
train_data = read_data('C:\\Users\\jshwa\\Downloads\\ratings_train.txt')
test_data = read_data('C:\\Users\\jshwa\\Downloads\\ratings_test.txt')

In [11]:
print(len(train_data))
print(len(train_data[0]))
print(len(test_data))
print(len(test_data[0]))

150000
3
50000
3


In [12]:
start = time()
import konlpy
from konlpy.tag import Okt
from konlpy.utils import pprint
pos_tagger = Okt()
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data]

end = time()
print('Time: {:f}s'.format(end-start))

C:\Users\jshwa\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


Time: 489.943608s


In [13]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')


tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_docs]

In [20]:
from gensim.models import doc2vec
doc_vectorizer = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs,epochs=10,total_examples=doc_vectorizer.corpus_count)
    doc_vectorizer.alpha -= 0.002  
    doc_vectorizer.min_alpha = doc_vectorizer.alpha

In [21]:
train_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
train_y = [doc.tags[0] for doc in tagged_train_docs]
test_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
test_y = [doc.tags[0] for doc in tagged_test_docs]

In [22]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)
classifier.score(test_x, test_y)

0.67726

In [23]:
from sklearn.neural_network import MLPClassifier

In [28]:
model = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=30,
    alpha=1e-4,
    solver='adam',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=0.01
)

In [32]:
start = time()
model.fit(train_x, train_y)
y_pred = model.predict(test_x)
acc = accuracy_score(y_pred, test_y)
end = time()
print("테스트 정확도: {:.2f}%".format(acc*100))
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.60085438
Iteration 2, loss = 0.57894384
Iteration 3, loss = 0.57065608
Iteration 4, loss = 0.56575273
Iteration 5, loss = 0.56176644
Iteration 6, loss = 0.55803160
Iteration 7, loss = 0.55416094
Iteration 8, loss = 0.55142271
Iteration 9, loss = 0.54860672
Iteration 10, loss = 0.54612909
Iteration 11, loss = 0.54367273
Iteration 12, loss = 0.54187194
Iteration 13, loss = 0.53873423
Iteration 14, loss = 0.53760300
Iteration 15, loss = 0.53522622
Iteration 16, loss = 0.53347028
Iteration 17, loss = 0.53211238
Iteration 18, loss = 0.53025743
Iteration 19, loss = 0.52850564
Iteration 20, loss = 0.52736591
Iteration 21, loss = 0.52655419
Iteration 22, loss = 0.52567531
Iteration 23, loss = 0.52421873
Iteration 24, loss = 0.52363034
Iteration 25, loss = 0.52326937
Iteration 26, loss = 0.52206635
Iteration 27, loss = 0.52110613
Iteration 28, loss = 0.51983560
Iteration 29, loss = 0.51967516
Iteration 30, loss = 0.51910215


C:\Users\jshwa\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


테스트 정확도: 70.06%
Time: 27.424693s
